In [1]:
# pip install pyfaidx biopython

### Inference

In [21]:
import torch
import pandas as pd
import numpy as np
import os
import json
from tqdm import tqdm
from pyfaidx import Fasta
from sklearn.metrics import confusion_matrix
from data_preparation import get_splam_official_logic_seq, one_hot_encode_splam
from metrics import compute_metrics

# --- CẤU HÌNH ---
GENOME_FA = r"D:\my_project\Bio_paper\Homo_sapiens.GRCh38.dna.primary_assembly.fa"
MODEL_PT = r"D:\my_project\Bio_paper\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SPLAM\pretrained_model\splam_script.pt"
DATA_FOLDER = r"D:\my_project\Bio_paper\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val"
OUTPUT_FOLDER = r"D:\my_project\Bio_paper\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SPLAM\results"

BATCH_SIZE = 256 
THRESHOLD = 0.21195 
TEST_FILES = ['test_1_1_1.csv', 'test_2_1_1.csv', 'test_4_1_1.csv', 'test_10_1_1.csv', 'test_data.csv']

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.jit.load(MODEL_PT).to(device)
model.eval()
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
genome = Fasta(GENOME_FA, sequence_always_upper=True)

for csv_file in TEST_FILES:
    input_path = os.path.join(DATA_FOLDER, csv_file)
    file_base = os.path.splitext(csv_file)[0]
    if not os.path.exists(input_path): continue
        
    print(f"\n🚀 Chạy Benchmark Final: {csv_file}")
    df = pd.read_csv(input_path)
    all_y_true, all_y_probs = [], []
    
    for i in tqdm(range(0, len(df), BATCH_SIZE)):
        batch_df = df.iloc[i : i + BATCH_SIZE]
        batch_seqs, batch_labels = [], []
        
        for _, row in batch_df.iterrows():
            parts = row['id'].split('_')
            chrom, pos, strand = parts[1], int(parts[2]), parts[3]
            label = int(row['Splicing_types'])
            seq = get_splam_official_logic_seq(chrom, pos, strand, label, genome)
            batch_seqs.append(one_hot_encode_splam(seq))
            batch_labels.append(label)
        
        input_tensor = torch.from_numpy(np.stack(batch_seqs)).to(device)
        # --- TRONG VÒNG LẶP INFERENCE ---
        with torch.no_grad():
            outputs = model(input_tensor)
            probs = torch.softmax(outputs, dim=1).cpu().numpy()
            
            for idx_in_batch, label in enumerate(batch_labels):
                # 1. TRÍCH XUẤT TÍN HIỆU DỰA TRÊN THỰC NGHIỆM CM:
                # Class 1 = Acceptor | Class 2 = Donor
                center = 200 if (label == 1 or label == 0) else 600

                window = range(center - 10, center + 11) # Kiểm tra +- 5bp
                
                # Max-pooling quanh 200 (Donor) và 600 (Acceptor)
                # Chúng ta gán đúng: p_don lấy từ lớp 2, p_acc lấy từ lớp 1
                p_don_raw = np.max(probs[idx_in_batch, 2, window]) 
                p_acc_raw = np.max(probs[idx_in_batch, 1, window])
                
                # Lấy xác suất nền tại chính tâm các site
                p_null_raw = np.max(probs[idx_in_batch, 0, window])
                
                # 2. CHUẨN HÓA: Mảng y_probs phải theo thứ tự y_true [Null, Donor, Acceptor]
                # Tức là: [Index 0, Index 1, Index 2]
                combined = np.array([p_null_raw, p_don_raw, p_acc_raw])
                combined /= (combined.sum() + 1e-9)
                
                all_y_true.append(label)
                all_y_probs.append(combined)

        # --- SAU VÒNG LẶP (PHẦN TÍNH TOÁN) ---
        y_true = np.array(all_y_true)
        y_probs = np.array(all_y_probs)
        y_preds = np.zeros(len(y_probs))

        for i in range(len(y_probs)):
            p_null, p_don, p_acc = y_probs[i] # p_don ở đây đã là Donor, p_acc đã là Acceptor
            
            if p_don > THRESHOLD or p_acc > THRESHOLD:
                # 1 là Donor, 2 là Acceptor (Khớp hoàn toàn với file CSV của bạn)
                y_preds[i] = 1 if p_don > p_acc else 2
            else:
                y_preds[i] = 0

        # Tính metrics
        results = compute_metrics(y_true, y_preds, probs=y_probs)
        results['confusion_matrix'] = confusion_matrix(y_true, y_preds).tolist()
    
    with open(os.path.join(OUTPUT_FOLDER, f"{file_base}_results.json"), 'w') as f:
        json.dump(results, f, indent=4)
        
    print(f"✅ Hoàn tất {file_base} | AUC: {results['auc']:.4f}")


🚀 Chạy Benchmark Final: test_1_1_1.csv


100%|██████████| 103/103 [00:20<00:00,  4.96it/s]


✅ Hoàn tất test_1_1_1 | AUC: 0.9475

🚀 Chạy Benchmark Final: test_2_1_1.csv


100%|██████████| 138/138 [00:26<00:00,  5.24it/s]


✅ Hoàn tất test_2_1_1 | AUC: 0.9426

🚀 Chạy Benchmark Final: test_4_1_1.csv


...

c:\Users\PC1\anaconda3\envs\splice_env\lib\site-packages\sklearn\metrics\_classification.py:2801: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
  2%|▏         | 63/3666 [00:10<10:22,  5.78it/s]c:\Users\PC1\anaconda3\envs\splice_env\lib\site-packages\sklearn\metrics\_classification.py:2801: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
  2%|▏         | 64/3666 [00:10<10:21,  5.80it/s]

c:\Users\PC1\anaconda3\envs\splice_env\lib\site-packages\sklearn\metrics\_classification.py:2801: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
  2%|▏         | 65/3666 [00:10<10:26,  5.75it/s]c:\Users\PC1\anaconda3\envs\splice_env\lib\site-packages\sklearn\metrics\_classification.py:2801: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
  2%|▏         | 66/3666 [00:10<10:26,  5.75it/s]

c:\Users\PC1\anaconda3\envs\splice_env\lib\site-packages\sklearn\metrics\_classification.py:2801: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
  2%|▏         | 67/3666 [00:10<10:27,  5.73it/s]c:\Users\PC1\anaconda3\envs\splice_env\lib\site-packages\sklearn\metrics\_classification.py:2801: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
  2%|▏         | 68/3666 [00:11<10:27,  5.73it/s]

100%|██████████| 3666/3666 [51:51<00:00,  1.18it/s]

✅ Hoàn tất test_data | AUC: 0.9331


In [8]:
import torch
import pandas as pd
import numpy as np
import os
import json
from tqdm import tqdm
from pyfaidx import Fasta
from collections import Counter
from data_preparation import get_splam_official_logic_seq, one_hot_encode_splam

# --- CẤU HÌNH ---
GENOME_FA = r"D:\my_project\Bio_paper\Homo_sapiens.GRCh38.dna.primary_assembly.fa"
MODEL_PT = r"D:\my_project\Bio_paper\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SPLAM\pretrained_model\splam_script.pt"
DATA_FOLDER = r"D:\my_project\Bio_paper\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val"
TEST_FILES = ['test_1_1_1.csv', 'test_2_1_1.csv', 'test_4_1_1.csv', 'test_10_1_1.csv', 'test_data.csv']
BATCH_SIZE = 128

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.jit.load(MODEL_PT).to(device)
model.eval()
genome = Fasta(GENOME_FA, sequence_always_upper=True)

peak_report = {}

for csv_file in TEST_FILES:
    input_path = os.path.join(DATA_FOLDER, csv_file)
    if not os.path.exists(input_path): continue
    
    print(f"\n🔍 Đang quét Peak cho file: {csv_file}")
    df = pd.read_csv(input_path)
    
    # Chỉ lấy các mẫu là Donor (1) hoặc Acceptor (2) để tìm peak
    df_sites = df[df['Splicing_types'].isin([1, 2])].copy()
    
    results = {"Donor_Peaks": [], "Acceptor_Peaks": []}
    
    for i in tqdm(range(0, len(df_sites), BATCH_SIZE)):
        batch_df = df_sites.iloc[i : i + BATCH_SIZE]
        batch_seqs = []
        batch_info = []
        
        for _, row in batch_df.iterrows():
            parts = row['id'].split('_')
            chrom, pos, strand = parts[1], int(parts[2]), parts[3]
            label = int(row['Splicing_types'])
            
            seq = get_splam_official_logic_seq(chrom, pos, strand, label, genome)
            batch_seqs.append(one_hot_encode_splam(seq))
            batch_info.append(label)
            
        input_tensor = torch.from_numpy(np.stack(batch_seqs)).to(device)
        
        with torch.no_grad():
            outputs = model(input_tensor)
            probs = torch.softmax(outputs, dim=1).cpu().numpy()
            
            for idx_in_batch, label in enumerate(batch_info):
                # Lớp 2 cho Donor, Lớp 1 cho Acceptor (theo swap của bạn)
                model_class = 2 if label == 1 else 1
                
                # Tìm vị trí cao nhất trên toàn bộ 800nt
                peak_pos = np.argmax(probs[idx_in_batch, model_class, :])
                
                if label == 1:
                    results["Donor_Peaks"].append(int(peak_pos))
                else:
                    results["Acceptor_Peaks"].append(int(peak_pos))
                    
    # Tổng hợp thống kê cho file này
    don_counts = Counter(results["Donor_Peaks"])
    acc_counts = Counter(results["Acceptor_Peaks"])
    
    peak_report[csv_file] = {
        "Donor": {
            "Top_3_Peaks": don_counts.most_common(3),
            "On_Target_200": don_counts.get(200, 0),
            "Total_Samples": len(results["Donor_Peaks"])
        },
        "Acceptor": {
            "Top_3_Peaks": acc_counts.most_common(3),
            "On_Target_600": acc_counts.get(600, 0),
            "Total_Samples": len(results["Acceptor_Peaks"])
        }
    }

# --- IN BÁO CÁO TỔNG HỢP ---
print("\n" + "="*50)
print("BÁO CÁO TỔNG HỢP VỊ TRÍ PEAK (ALIGNMENT REPORT)")
print("="*50)
for file, data in peak_report.items():
    print(f"\n📂 File: {file}")
    d = data["Donor"]
    a = data["Acceptor"]
    print(f"  [Donor]    - On-Target (200): {d['On_Target_200']}/{d['Total_Samples']} ({d['On_Target_200']/d['Total_Samples']*100:.1f}%)")
    print(f"             - Top Peaks: {d['Top_3_Peaks']}")
    print(f"  [Acceptor] - On-Target (600): {a['On_Target_600']}/{a['Total_Samples']} ({a['On_Target_600']/a['Total_Samples']*100:.1f}%)")
    print(f"             - Top Peaks: {a['Top_3_Peaks']}")

# Lưu báo cáo vào file JSON
with open("peak_alignment_report.json", "w") as f:
    json.dump(peak_report, f, indent=4)


🔍 Đang quét Peak cho file: test_1_1_1.csv


100%|██████████| 137/137 [00:13<00:00, 10.50it/s]



🔍 Đang quét Peak cho file: test_2_1_1.csv


100%|██████████| 137/137 [00:12<00:00, 11.16it/s]



🔍 Đang quét Peak cho file: test_4_1_1.csv


...


🔍 Đang quét Peak cho file: test_10_1_1.csv


100%|██████████| 137/137 [00:08<00:00, 15.40it/s]



🔍 Đang quét Peak cho file: test_data.csv


100%|██████████| 137/137 [00:08<00:00, 15.48it/s]


BÁO CÁO TỔNG HỢP VỊ TRÍ PEAK (ALIGNMENT REPORT)

📂 File: test_1_1_1.csv
  [Donor]    - On-Target (200): 5002/8822 (56.7%)
             - Top Peaks: [(200, 5002), (600, 1652), (410, 130)]
  [Acceptor] - On-Target (600): 6308/8666 (72.8%)
             - Top Peaks: [(600, 6308), (200, 1596), (398, 32)]

📂 File: test_2_1_1.csv
  [Donor]    - On-Target (200): 5002/8822 (56.7%)
             - Top Peaks: [(200, 5002), (600, 1652), (410, 130)]
  [Acceptor] - On-Target (600): 6308/8666 (72.8%)
             - Top Peaks: [(600, 6308), (200, 1596), (398, 32)]

📂 File: test_4_1_1.csv
  [Donor]    - On-Target (200): 5002/8822 (56.7%)
             - Top Peaks: [(200, 5002), (600, 1652), (410, 130)]
  [Acceptor] - On-Target (600): 6308/8666 (72.8%)
             - Top Peaks: [(600, 6308), (200, 1596), (398, 32)]

📂 File: test_10_1_1.csv
  [Donor]    - On-Target (200): 5002/8822 (56.7%)
             - Top Peaks: [(200, 5002), (600, 1652), (410, 130)]
  [Acceptor] - On-Target (600): 6308/8666 (72.8%)
  